# Import Molecule from ASE into AiiDA

In [3]:
from __future__ import print_function

#from aiida import load_dbenv, is_dbenv_loaded
#from aiida.backends import settings
#if not is_dbenv_loaded():
#    load_dbenv(profile=settings.AIIDADB_PROFILE)

#from aiida.orm.data.structure import StructureData

import ase.build
from ase.collections import g2

import ipywidgets as ipw
from IPython.display import display, clear_output
#import nglview
import StringIO

ImportError: No module named 'StringIO'

In [ ]:
viewer = nglview.NGLWidget()
coords = ipw.HTML()
display(ipw.VBox([viewer, coords]))

In [ ]:
def update_view():
    global atoms
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()
       
    tmp = StringIO.StringIO()
    atoms.write(tmp, format="xyz")
    coords.value = "<pre>"+tmp.getvalue()+"</pre>"
    tmp.close()

## Step 1: Select Molecule

In [ ]:
def on_mol_change(c):
    global atoms
    atoms = ase.build.molecule(inp_mol.value)
    update_view()
    
inp_mol = ipw.Dropdown(options=g2.names, value="H2O")
inp_mol.observe(on_mol_change, names='value')
display(inp_mol)
on_mol_change(None)

## Step 2: Define Cell

In [ ]:
def on_click_vac(b):
    global atoms
    atoms.center(vacuum=inp_vac.value)
    atoms.pbc = (True,True,True)
    update_view()

inp_vac = ipw.FloatText(value=2.5)
lab_vac = ipw.Label(u'Vacuum [Å]:')
btn_vac = ipw.Button(description="Add Vacuum")
btn_vac.on_click(on_click_vac)
display(ipw.HBox([lab_vac, inp_vac, btn_vac]))

## Step 3: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        s = StructureData(ase=atoms)
        s.description = inp_descr.value
        s.store()
        print("Stored in AiiDA: "+repr(s))

store_out = ipw.Output()
inp_descr = ipw.Text(placeholder="Description (optional)")
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]), store_out)